In [1]:
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML
import pandas as pd
import random
import copy

In [2]:
np.linspace(-3,3,np.random.randint(10,50))

array([-3. , -2.7, -2.4, -2.1, -1.8, -1.5, -1.2, -0.9, -0.6, -0.3,  0. ,
        0.3,  0.6,  0.9,  1.2,  1.5,  1.8,  2.1,  2.4,  2.7,  3. ])

In [3]:

class ThreeLinkArm:
    
    def __init__(self):
        # Set parameters for the 3-link planar arm
        self.theta1 = np.deg2rad(0)
        self.theta2 = np.deg2rad(90)
        self.theta3 = np.deg2rad(90)
        self.orig_l1 = np.random.randint(4,9)
        self.orig_l2 = np.random.randint(4,9)
        self.orig_l3 = np.random.randint(4,9)
        self.l1=copy.copy(self.orig_l1)
        self.l2=copy.copy(self.orig_l2)
        self.l3=copy.copy(self.orig_l3)
        max_len_shift=int(np.min([self.l1,self.l2,self.l3])*(2/3))
        self.arr1=np.linspace(-max_len_shift,max_len_shift,np.random.randint(10,50))
        self.arr1=np.append(self.arr1,np.flip(self.arr1))
        self.permute=np.random.permutation([0.5,0.5,-1])

    def forwardKinematics(self, theta1, theta2, theta3):
        # Define the homogeneous transformation matrices for the 3-link planar arm
        self.theta1 = theta1
        self.theta2 = theta2
        self.theta3 = theta3
        self.t01 = np.matrix([[np.cos(self.theta1), -np.sin(self.theta1), 0], [np.sin(self.theta1), np.cos(self.theta1), 0], [0, 0, 1]])
        self.t12 = np.matrix([[np.cos(self.theta2), -np.sin(self.theta2), self.l1], [np.sin(self.theta2), np.cos(self.theta2), 0], [0, 0, 1]])
        self.t23 = np.matrix([[np.cos(self.theta3), -np.sin(self.theta3), self.l2], [np.sin(self.theta3), np.cos(self.theta3), 0], [0, 0, 1]])        
        self.t3end = np.matrix([[np.cos(self.theta3), -np.sin(self.theta3), self.l3], [np.sin(self.theta3), np.cos(self.theta3), 0], [0, 0, 1]])
        self.t0end = self.t01*self.t12*self.t23*self.t3end
        return self.t0end
    
    def findJointPos(self): 
        # Find the x,y position of each joint and end effector so it can be plotted
        # Find the transformation matrices for joint 2 and joint 3
        self.t02 = self.t01*self.t12
        self.t03 = self.t01*self.t12*self.t23
        # Find the x, y coordinates for joints 2 and 3. Put them in a list j2 = [x,y]
        j2 = [ self.t02[0, 2], self.t02[1, 2] ]
        j3 = [ self.t03[0, 2], self.t03[1, 2] ]
        endeff = [self.t0end[0,2],self.t0end[1,2]]
        
        self.jnt2pos, self.jnt3pos, self.endEffPos = j2,j3,endeff
        return j2,j3,endeff
    
    def geomJacobian(self, jnt2pos=None, jnt3pos=None, endEffPos=None):
        if jnt2pos is None:
            jnt2pos=self.jnt2pos 
        if jnt3pos is None:
            jnt3pos=self.jnt3pos
        if endEffPos is None:
            endEffPos=self.endEffPos
        ai = np.array([0,0,1])
        col0 = np.array(endEffPos + [0])
        col1 = np.array(endEffPos + [0]) - np.array(jnt2pos + [0])
        col2 = np.array(endEffPos + [0]) - np.array(jnt3pos + [0])
        J = np.array([np.cross(ai,col0), np.cross(ai,col1), np.cross(ai,col2)]).T 
        return J
    
    def random_link_len_change(self,i):
#         self.l1=self.orig_l1+np.random.uniform(-1,1)
#         self.l2=self.orig_l2+np.random.uniform(-1,1)
#         self.l3=self.orig_l3+np.random.uniform(-1,1)

#         self.l1=self.orig_l1+np.random.choice([np.random.normal(1,0.5),np.random.normal(-1,0.5)])
#         self.l2=self.orig_l2+np.random.choice([np.random.normal(1,0.5),np.random.normal(-1,0.5)])
#         self.l3=self.orig_l3+np.random.choice([np.random.normal(1,0.5),np.random.normal(-1,0.5)])
        
        self.l1=self.orig_l1+self.permute[0]*self.arr1[i%len(self.arr1)]
        self.l2=self.orig_l2+self.permute[1]*self.arr1[i%len(self.arr1)]
        self.l3=self.orig_l3+self.permute[2]*self.arr1[i%len(self.arr1)]

In [4]:
def plotArm(jnt2pos, jnt3pos, endEffectPos, target=None, step=None):
    # set up figure
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, autoscale_on=False, xlim=(-25, 25), ylim=(-25, 25))
    ax.grid()
    
    if step is not None:
        step = np.array(step)
        plt.scatter(step[:, 0],step[:, 1], color='lightblue', marker='o')
        plt.plot(step[:, 0],step[:, 1], color='lightblue')
    if target is not None:
        target = np.array(target)
        plt.scatter(target[:, 0],target[:, 1], color='red', marker='o')
        
    line, = ax.plot([], [], 'o-', lw=4, mew=5)
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    line.set_data([], [])
    time_text.set_text('')
    x = np.array([0, jnt2pos[0], jnt3pos[0], endEffectPos[0]])
    y = np.array([0, jnt2pos[1], jnt3pos[1], endEffectPos[1]])
    line.set_data((x,y))
    
    plt.show()
    
# gif animation plotter
def plotArm2(jnt2pos, jnt3pos, endEffectPos,x1,y1,x2,y2, target=None, step=None, fig=None, camera=None):
    # set up figure
    ax = fig.add_subplot(111, autoscale_on=False,
                         xlim=(-25, 25), ylim=(-25, 25))
    ax.grid()
    line, = ax.plot([], [], 'o-', lw=4, mew=5, color='lightblue')
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    line.set_data([], [])
    time_text.set_text('')
    x = np.array([0, jnt2pos[0], jnt3pos[0], endEffectPos[0]])
    y = np.array([0, jnt2pos[1], jnt3pos[1], endEffectPos[1]])
    line.set_data((x,y))

    if(not np.isnan(x1[0])):
            plt.plot([jnt2pos[0],jnt3pos[0],x1[0],jnt2pos[0]],
                     [jnt2pos[1],jnt3pos[1],y1[0],jnt2pos[1]],'o-', lw=2, mew=5, color='green')
    if(not np.isnan(x1[1])):
            plt.plot([jnt3pos[0],endEffectPos[0],x1[1],jnt3pos[0]],
                     [jnt3pos[1],endEffectPos[1],y1[1],jnt3pos[1]],'o-', lw=2, mew=5, color='yellow')
    
    plt.plot([endEffectPos[0],x2[0]],[endEffectPos[1],y2[0]],'o-', lw=2, mew=5, color='lightblue')
    plt.plot([endEffectPos[0],x2[1]],[endEffectPos[1],y2[1]],'o-', lw=2, mew=5, color='lightblue')


    if step is not None:
        step = np.array(step)
        plt.scatter(step[:, 0],step[:, 1], color='lightblue', marker='o')
        plt.plot(step[:, 0],step[:, 1], color='lightblue')
    if target is not None:
        target = np.array(target)
        plt.scatter(target[:, 0],target[:, 1], color='red', marker='o')

    camera.snap()

In this part, let's plan some simple polynomial trajectories in the work space. 

First, setup a simple environment with the arm manipulator and four intermediate interpolation points.

In [5]:
def create_simulation(animation_index,samples):
    
    # Setup our new manipulator
    arm = ThreeLinkArm()
    
    # define the initial configuration of the manipulator (you are welcomed to try other values)
    initTheta = [np.deg2rad(0),np.deg2rad(45),np.deg2rad(-45)]

    # Initialise the arm with the initial values
    _ = arm.forwardKinematics(*initTheta)

    # Get the initial end-effector position 
    init_joint2pos, init_joint3pos, EF_init_pos = arm.findJointPos()

    # Define the target postions of the end-effector
    # EF_target_pos = np.array([[EF_init_pos[0], 6.,2.,-4.,-8.],
    #                           [EF_init_pos[1], -6.,8.,-5.,6.]]).T
    ##Choose points randomly, choose x equally spaced and y randomly
    L=arm.l1+arm.l2+arm.l3
    print(L)
    x_points=np.array([2/3*L, 1/3*L, -1/3*L, -2/3*L])

    y_points=[]
    R=3*L/4
    for i in range(4):
        y_points.append(random.uniform(-1*np.sqrt(R**2-x_points[i]**2),
                                          np.sqrt(R**2-x_points[i]**2)))
    y_points=np.array(y_points)
    
    EF_target_pos = np.array([[EF_init_pos[0], x_points[0],x_points[1],x_points[2],x_points[3]],
                              [EF_init_pos[1], y_points[0],y_points[1],y_points[2],y_points[3]]]).T

    # Plot the pose of the arm
    #plotArm(*arm.findJointPos(), target=EF_target_pos)
    print(f"The interpolation points are stored in the variable \"EF_target_pos\": \n{EF_target_pos}")
    
    pol = np.polyfit(EF_target_pos[:, 0], EF_target_pos[:, 1], EF_target_pos.shape[0]-1)  # polynomial coefficients
    
    # Plotting the planned trajectory
    #epsilon = 100
    epsilon = samples
    
    xx = np.linspace(EF_target_pos[0,0],EF_target_pos[-1,0], epsilon)
    yy = np.polyval(pol,xx)   
    curve = np.hstack([xx[:, None], yy[:, None]]) 
    
    interpolated_points=curve

#     plotArm(*arm.findJointPos(), target=EF_target_pos, step=interpolated_points)
    print(f"The intermediate points are stored in the variable \"interpolated_points\"")
    
    # set the current E-F position for the IK loop
    joint2pos, joint3pos, endEffectorPos = init_joint2pos, init_joint3pos, EF_init_pos
    # set the initial robot configuration
    newTheta = initTheta

    # Set up the animation generator
    fig = plt.figure(figsize=(10,10))
    camera = Camera(fig)

    temp=np.random.randint(0,2,2)
    temp_angle=np.random.randint(15,60,2) 
    print(temp)
    print(temp_angle)
    
    temp_eeff_angle=np.append(np.linspace(0,45,int(interpolated_points.shape[0]/8)),
                          np.linspace(45,0,int(interpolated_points.shape[0]/8)))
    
    eeff_len_orig=np.random.randint(2,5)
    
    keypoints_data=pd.DataFrame(columns=range((4+np.sum(temp)+2)*2),dtype=float)
    
    arr2=np.linspace(-1,1,np.random.randint(10,50))
    arr3=np.linspace(-int(eeff_len_orig/2),int(eeff_len_orig/2),np.random.randint(10,50))
    
    arr2=np.append(arr2,np.flip(arr2))
    arr3=np.append(arr3,np.flip(arr3))

    for i in range(interpolated_points.shape[0]):
        J = arm.geomJacobian()

        newgoal = interpolated_points[i, :]
        deltaStep = newgoal - endEffectorPos
        subtarget = np.array([deltaStep[0], deltaStep[1], 0]) 

        radTheta = np.linalg.pinv(J) @ subtarget
        newTheta = newTheta + radTheta

        # ----------- Do forward kinematics to plot the arm ---------------
        arm.random_link_len_change(i)
        eeff_len=eeff_len_orig+arr3[i%len(arr3)]
        _ = arm.forwardKinematics(newTheta[0],newTheta[1],newTheta[2])
        joint2pos, joint3pos, endEffectorPos = arm.findJointPos()

        x1=[]
        y1=[]
        x2=[]
        y2=[]

        if(temp[0]==1):
            x1=x1+[arm.l1*np.cos(newTheta[0]) \
                   +(3+arr2[i%len(arr2)])*np.cos(newTheta[0]+newTheta[1]+np.deg2rad(temp_angle[0]))]
            y1=y1+[arm.l1*np.sin(newTheta[0]) \
                   +(3+arr2[i%len(arr2)])*np.sin(newTheta[0]+newTheta[1]+np.deg2rad(temp_angle[0]))]
        else:
            x1=x1+[np.nan]
            y1=y1+[np.nan]

        if(temp[1]==1):
            x1=x1+[arm.l1*np.cos(newTheta[0]) \
                   +arm.l2*np.cos(newTheta[0]+newTheta[1]) \
                   +(3+arr2[i%len(arr2)])*np.cos(newTheta[0]+newTheta[1]+newTheta[2]+np.deg2rad(temp_angle[1]))]
            y1=y1+[arm.l1*np.sin(newTheta[0]) \
                   +arm.l2*np.sin(newTheta[0]+newTheta[1]) \
                   +(3+arr2[i%len(arr2)])*np.sin(newTheta[0]+newTheta[1]+newTheta[2]+np.deg2rad(temp_angle[1]))]
        else:
            x1=x1+[np.nan]
            y1=y1+[np.nan]

        x2=x2+[arm.l1*np.cos(newTheta[0]) \
               +arm.l2*np.cos(newTheta[0]+newTheta[1]) \
               +arm.l3*np.cos(newTheta[0]+newTheta[1]+newTheta[2]) \
               +eeff_len*np.cos(newTheta[0]+newTheta[1]+newTheta[2] \
                                +np.deg2rad(temp_eeff_angle[i%len(temp_eeff_angle)]))]
    
        y2=y2+[arm.l1*np.sin(newTheta[0]) \
               +arm.l2*np.sin(newTheta[0]+newTheta[1]) \
               +arm.l3*np.sin(newTheta[0]+newTheta[1]+newTheta[2]) \
               +eeff_len*np.sin(newTheta[0]+newTheta[1]+newTheta[2] \
                                +np.deg2rad(temp_eeff_angle[i%len(temp_eeff_angle)]))]
        x2=x2+[arm.l1*np.cos(newTheta[0]) \
               +arm.l2*np.cos(newTheta[0]+newTheta[1]) \
               +arm.l3*np.cos(newTheta[0]+newTheta[1]+newTheta[2]) \
               +eeff_len*np.cos(newTheta[0]+newTheta[1]+newTheta[2] \
                                -np.deg2rad(temp_eeff_angle[i%len(temp_eeff_angle)]))]
    
        y2=y2+[arm.l1*np.sin(newTheta[0]) \
               +arm.l2*np.sin(newTheta[0]+newTheta[1]) \
               +arm.l3*np.sin(newTheta[0]+newTheta[1]+newTheta[2]) \
               +eeff_len*np.sin(newTheta[0]+newTheta[1]+newTheta[2] \
                                -np.deg2rad(temp_eeff_angle[i%len(temp_eeff_angle)]))]

        temp_keypoints=np.array([0,0,joint2pos[0],joint2pos[1],joint3pos[0],joint3pos[1],
                                 endEffectorPos[0],endEffectorPos[1]])
        for k in range(len(temp)):
            if(temp[k]!=0):
                temp_keypoints=np.append(temp_keypoints,np.array([x1[k],y1[k]]))
        
        temp_keypoints=np.append(temp_keypoints,np.array([x2[0],y2[0],x2[1],y2[1]]))
        
        keypoints_data = keypoints_data.append(pd.DataFrame(temp_keypoints[None,:],columns=keypoints_data.columns),
                                               ignore_index = True)
        
        plotArm2(joint2pos, joint3pos, endEffectorPos,x1,
                 y1,x2,y2, target=EF_target_pos, fig=fig, camera=camera)


    plt.close()
    animation = camera.animate()
    animation.save('./animated_motion_dynamic_len/animated_motion'+str(animation_index)+'.gif', writer = 'imagemagick')
    
    # create the adjacency matrix
    adj_mat_size=sum(np.invert(np.isnan(x1)))
    adj_mat=np.zeros((4+adj_mat_size+2,4+adj_mat_size+2))
    adj_mat[0][1]=1
    adj_mat[1][2]=1
    adj_mat[2][3]=1
    c2=1
    for i in range(len(x1)):
        if(not np.isnan(x1[i])):
            adj_mat[i+1][3+c2]=1
            adj_mat[i+2][3+c2]=1
            c2=c2+1
    adj_mat[3][3+c2]=1
    adj_mat[3][3+c2+1]=1
    
    pd.DataFrame(adj_mat).astype(int).to_csv('./animated_motion_dynamic_len/adj_mat'+str(animation_index)+'.csv')
    keypoints_data.to_csv('./animated_motion_dynamic_len/ts_data'+str(animation_index)+'.csv')    
    

In [6]:
import os
os.environ['MAGICK_THREAD_LIMIT'] = '2'

In [7]:
# create_simulation(1020,50)

In [8]:
for i in range(125):
    create_simulation(i,50)

16
The interpolation points are stored in the variable "EF_target_pos": 
[[ 14.82842712   2.82842712]
 [ 10.66666667  -1.53630152]
 [  5.33333333  -3.93677208]
 [ -5.33333333  -0.17048685]
 [-10.66666667  -0.71639453]]
The intermediate points are stored in the variable "interpolated_points"
[0 1]
[59 38]


<ipython-input-4-99abad6b2bba>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,


15
The interpolation points are stored in the variable "EF_target_pos": 
[[ 13.53553391   3.53553391]
 [ 10.           2.39725507]
 [  5.           0.77032441]
 [ -5.           8.51713258]
 [-10.           2.58957474]]
The intermediate points are stored in the variable "interpolated_points"
[1 0]
[18 28]


<ipython-input-4-99abad6b2bba>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,


17
The interpolation points are stored in the variable "EF_target_pos": 
[[ 14.94974747   4.94974747]
 [ 11.33333333   5.77772319]
 [  5.66666667  -8.47903295]
 [ -5.66666667   9.73780157]
 [-11.33333333  -4.03755025]]
The intermediate points are stored in the variable "interpolated_points"
[0 1]
[31 20]


...

<ipython-input-4-99abad6b2bba>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,


17
The interpolation points are stored in the variable "EF_target_pos": 
[[ 15.53553391   3.53553391]
 [ 11.33333333   5.12154086]
 [  5.66666667  -4.68376222]
 [ -5.66666667  10.60853192]
 [-11.33333333  -1.70894357]]
The intermediate points are stored in the variable "interpolated_points"
[0 0]
[54 54]


<ipython-input-4-99abad6b2bba>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,


20
The interpolation points are stored in the variable "EF_target_pos": 
[[ 18.82842712   2.82842712]
 [ 13.33333333  -1.02046507]
 [  6.66666667   8.02313857]
 [ -6.66666667  -8.89833444]
 [-13.33333333   2.96544192]]
The intermediate points are stored in the variable "interpolated_points"
[1 1]
[52 18]


<ipython-input-4-99abad6b2bba>:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.add_subplot(111, autoscale_on=False,
